In [1]:
!pip install datasets
!pip install transformers 
!pip install evaluate
!pip install rouge-score
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from google.colab import drive
from transformers import pipeline
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import datasets
import re
import time
import math
import evaluate
import torch
import numpy as np
import pandas as pd
import ast

# Read in files and subset

In [3]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%ls

gdrive/  sample_data/


In [5]:
# read in text and summaries
all_text = pd.read_csv("/content/gdrive/MyDrive/modified_data/text_and_summaries_filtered.csv")
all_text.head(-20)

,state_name,state,bill_id,bill_name,original_text,cleaned_text,split_cleaned_text,summary,summary_source,category,status,link
0,Alaska,AK,AK HB27,HB27,HB0027a 1 HB 27 New Text Underlined DEL...,. section . as . is amended by adding new sub...,"['section', 'as', 'is amended by adding new s...",The bill amends an existing regulation of scho...,handwritten,Schools & Education,"Referred to committee, 01/19/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...
1,Alaska,AK,AK HB105,HB105,HB0105a 1 HB 105 New Text Underlined DE...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...
2,Alaska,AK,AK SB96,SB96,SB0096A 1 SB 96 New Text Underlined DEL...,. section . as . is amended to read . local...,"['section', 'as', 'is amended to read', 'local...",This bill adds to the list of parental rights ...,fast democracy,Schools & Education,"First read and referred to committee, 03/08/2023",https://www.akleg.gov/basis/Bill/Detail/33?Roo...
3,Arizona,AZ,AZ SB1028,SB1028,i Senate Engrossed adult cabaret pe...,. section . title . chapter . arizona revised ...,"['section', 'title', 'chapter', 'arizona revis...",This bill prohibits a person or business from ...,fast democracy,Free Speech & Expression,Passed Senate; House Committee of the Whole pa...,https://apps.azleg.gov/BillStatus/BillOverview...
4,Arizona,AZ,AZ SB1026,SB1026,i Senate Engrossed state monies dr...,. section . title . chapter . article . arizon...,"['section', 'title', 'chapter', 'article', 'ar...",This bill prohibits the use of state monies an...,fast democracy,Free Speech & Expression,Passed Senate; House Committee of the Whole pa...,https://apps.azleg.gov/BillStatus/BillOverview...
...,...,...,...,...,...,...,...,...,...,...,...,...
320,Virginia,VA,VA SB962,SB962,2023SESSION INTRODUCED 23101004D 1 SENATE BILL...,. that the code of virginia is amended by addi...,['that the code of virginia is amended by addi...,This bill requires each public elementary or s...,fast democracy,Schools & Education,"Passed by indefinitely by committee, 02/02/2023",https://lis.virginia.gov/cgi-bin/legp604.exe?2...
321,Virginia,VA,VA SB911,SB911,2023SESSION INTRODUCED 23102088D 1 SENATE BILL...,. that the code of virginia is amended by addi...,['that the code of virginia is amended by addi...,This bill requires each public elementary or s...,fast democracy,Schools & Education,"Incorporated into SB 1186, 02/02/2023",https://lis.virginia.gov/cgi-bin/legp604.exe?2...
322,Virginia,VA,VA HB1707,HB1707,2023SESSION INTRODUCED 23101240D 1 HOUSEBILLNO...,. that . of the code of virginia is amended an...,"['that', 'of the code of virginia is amended a...",This bill requires any person licensed as admi...,fast democracy,Schools & Education,"Incorporated by committee into HB 2432, 02/03/...",https://lis.virginia.gov/cgi-bin/legp604.exe?2...
323,Virginia,VA,VA SB1203,SB1203,2023SESSION INTRODUCED 23103055D 1 SENATE BILL...,. that the code of virginia is amended by addi...,['that the code of virginia is amended by addi...,This bill establishes the Children Deserve Hel...,fast democracy,Healthcare,"Incorporated by committee into SB 791, 02/02/2023",https://lis.virginia.gov/cgi-bin/legp604.exe?2...


In [6]:
cols_keep = ['state','bill_id', 'summary', 'split_cleaned_text']

all_text = all_text[cols_keep]
# join the split text together
all_text['split_text'] = all_text['split_cleaned_text'].apply(lambda x: ' '.join(ast.literal_eval(x)))

all_text.rename({'bill_id':'title',
                 'split_text':'text',
                }, axis='columns', inplace=True)
all_text.head()

<ipython-input-6-26d40715aa12>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_text['split_text'] = all_text['split_cleaned_text'].apply(lambda x: ' '.join(ast.literal_eval(x)))
<ipython-input-6-26d40715aa12>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_text.rename({'bill_id':'title',


,state,title,summary,split_cleaned_text,text
0,AK,AK HB27,The bill amends an existing regulation of scho...,"['section', 'as', 'is amended by adding new s...",section as is amended by adding new subsectio...
1,AK,AK HB105,This bill adds to the list of parental rights ...,"['section', 'as', 'is amended to read', 'local...",section as is amended to read local school boa...
2,AK,AK SB96,This bill adds to the list of parental rights ...,"['section', 'as', 'is amended to read', 'local...",section as is amended to read local school boa...
3,AZ,AZ SB1028,This bill prohibits a person or business from ...,"['section', 'title', 'chapter', 'arizona revis...",section title chapter arizona revised statutes...
4,AZ,AZ SB1026,This bill prohibits the use of state monies an...,"['section', 'title', 'chapter', 'article', 'ar...",section title chapter article arizona revised ...


## Split into training and testing

In [7]:
train, test = train_test_split(all_text[['text', 'summary', 'title']], test_size = 0.2)
print(len(train))
print(len(test))


276
69


In [8]:
train.to_csv("/content/gdrive/MyDrive/modified_data/train-not-split_bills.csv")
test.to_csv("/content/gdrive/MyDrive/modified_data/test-not-split_bills.csv")

# Create HuggingFace Objects

In [9]:
train_dataset = datasets.Dataset.from_dict(train)
test_dataset = datasets.Dataset.from_dict(test)
billsum = datasets.DatasetDict({"train": train_dataset, "test": test_dataset})

# Data Loading

In [10]:
billsum['test'][0]['summary']

"As introduced, this bill clarifies that prescribing hormone treatment for minors is not a standard medical practice when the treatment is for the purpose of enabling a minor to identify with, or live as, a purported identity inconsistent with the minor’s sex or treating purported discomfort or distress from a discordance between a minor's sex and asserted identity."

# Load Tokenizer

In [11]:
## PREPROCESSING
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [12]:
#The preprocessing function you want to create needs to:

# 1. Prefix the input with a prompt so T5 knows this is a summarization task. 
#    Some models capable of multiple NLP tasks require prompting for specific tasks.
# 2. Use the keyword text_target argument when tokenizing labels.
# 3. Truncate sequences to be no longer than the maximum length set by the max_length parameter.

prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    # Tokenizes and truncates
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# To apply the preprocessing function over the entire dataset
# You can speed up the map function by setting batched=True to process multiple elements of the dataset at once
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/276 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum['train']

In [ ]:
# Create batch of examples using DataCollatorForSeq2Seq. It's better to dynamically pad the sentences to the longest length
# in a batch during collation instead of padding to the model's maximum length
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Set Up Evaluation

In [ ]:
## SET UP EVALUATOR

# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Create function that passes predictions and labels to compute the ROUGE metric
# This is what is used for model training
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train Model

In [ ]:
#model.save_weights("/content/gdrive/My Drive/weights.h5")

In [ ]:
%ls

In [ ]:
output_directory = "/content/gdrive/My Drive/modified_data"
model_directory =  "/content/gdrive/My Drive/summarizer_model_not-split"

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
#     output_dir="/content/gdrive/My Drive/my_awesome_billsum_model",
    output_dir = output_directory,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
#     fp16=True,
    push_to_hub=False, # changed to false
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# start = time.time()
# trainer.train()
# end = time.time()
# print((end - start)/60)

In [ ]:
# Save the model
trainer.save_model(model_directory)

# Test on all bills

In [ ]:
len(all_text)

345

In [ ]:
# build the summarizer pipeline
summarizer = pipeline("summarization", model=str(model_directory))

# run the summarizer on every row
all_text.loc[:, 'model_summary'] = all_text.loc[:,'text'].apply(lambda x: summarizer(x)[0]['summary_text'])

In [ ]:
model_summaries_df.to_csv("modified_data/model_summaries-not-split.csv")